In [ ]:
import os
import copy
from collections import deque
from enum import Enum

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve
from torchinfo import summary

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import MultiStepLR

In [ ]:
RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

PREDICTION_WINDOW_SIZE = 5
OBSERVATIONS_FOLDER = "observations"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class Action(Enum):
    IDLE = 0
    PICK = 1
    PLACE = 2
    SCREW_WRENCH = 3

OBJECT_NAMES = {
    0: "small_screw",
    1: "big_screw",
    2: "small_wrench",
    3: "big_wrench",
    4: "cap",
    5: "barrel",
    6: "piston",
    7: "support",
    8: "air_connector",
    9: "nut"
}

num_classes_actions = len(Action)
num_classes_objects = len(OBJECT_NAMES) + 1 # Last class for empty hand

In [ ]:
base_df_path = os.path.join("assembling_sequence.csv")
base_df = pd.read_csv(base_df_path)

base_df.head()

In [ ]:
DELETATION_PROBABILITY = 0.05
IDLE_REPLACE_PROBABILITY = 0.05
IDLE_INSERT_PROBABILITY = 0.05
EMPTY_OBJECT_PROBABILITY = 0.05
RANDOM_OBJECT_PROBABILITY = 0.05

def generate_variation_df(df):
    rows = []

    for i, row in df.iterrows():
        deletation_probability = np.random.random()
        if deletation_probability > DELETATION_PROBABILITY:
            row_dict = row.to_dict()
            idle_replace_probability = np.random.random()
            if idle_replace_probability < IDLE_REPLACE_PROBABILITY:
                row_dict["action"] = 0
            object_probability = np.random.random()
            if object_probability < EMPTY_OBJECT_PROBABILITY:
                row_dict["right_hand_object"] = len(OBJECT_NAMES) if object_probability > RANDOM_OBJECT_PROBABILITY else np.random.randint(len(OBJECT_NAMES) - 1)
            rows.append(row_dict)
                
            idle_insert_probability = np.random.random()
            if idle_insert_probability < IDLE_INSERT_PROBABILITY and i < (len(df) - 1):
                idle_object = len(OBJECT_NAMES) if idle_insert_probability > RANDOM_OBJECT_PROBABILITY else np.random.randint(len(OBJECT_NAMES) - 1)
                idle_row = {"action": 0, "right_hand_object": idle_object}
                rows.append(idle_row)

    return pd.DataFrame(rows) 

In [ ]:
NUM_VARIATIONS = 20

if not os.path.exists(OBSERVATIONS_FOLDER):
    os.makedirs(OBSERVATIONS_FOLDER)

print(f"Saving {NUM_VARIATIONS} variations of base dataframe in folder {OBSERVATIONS_FOLDER}...")
for i in tqdm(range(NUM_VARIATIONS)):
    var_df_path = os.path.join(OBSERVATIONS_FOLDER, f"observation_{i}.csv")
    var_df = generate_variation_df(base_df)
    var_df.to_csv(var_df_path, encoding="utf-8", index=False)

In [ ]:
observation_dfs = []
observation_dfs.append(base_df)
for observation_csv in tqdm(os.listdir(OBSERVATIONS_FOLDER)):
    observation_df_path = os.path.join(OBSERVATIONS_FOLDER, observation_csv)
    observation_dfs.append(pd.read_csv(observation_df_path))

concatenated_df = pd.concat(observation_dfs, ignore_index=True)
print("concatenated_df shape:", concatenated_df.shape)

In [ ]:
def one_hot_encode_class(class_index, num_classes):
    one_hot_vector = np.zeros(num_classes, dtype=int)
    one_hot_vector[class_index] = 1
    return one_hot_vector

def df_to_X_y_data(df, window_size):
    df_np = df.to_numpy()
    X_data = []
    y1_data = []
    y2_data = []
    for i in range(len(df_np) - window_size):
        window_X = []
        for row in df_np[i:(i + window_size)]:
            window_X.append(np.concatenate((one_hot_encode_class(row[0], num_classes_actions), one_hot_encode_class(row[1], num_classes_objects)), axis=0))
            
        X_data.append(window_X)
        y1_data.append(one_hot_encode_class(df_np[i + window_size][0], num_classes_actions))
        y2_data.append(one_hot_encode_class(df_np[i + window_size][1], num_classes_objects))

    return np.array(X_data), [np.array(y1_data), np.array(y2_data)]

def multi_task_train_val_test_split(X, ys, val_size=0.1, test_size=0.1, shuffle=True, random_state=None):
    """
    Split the data into training and testing sets for multiple y arrays.
    
    Parameters:
        X: array-like, shape (n_samples, n_features)
            Input data.
        *ys: array-like, shape (n_samples, )
            Multiple target variables.
        val_size, test_size: float [0.0, 1.0), default=0.1
            Represents the proportion of the dataset to include in the validation and test split respectively.
        shuffle: bool, default=True
            Whether to shuffle the input dataset.
        random_state: int or None, default=None
            Controls the randomness of the shuffle.
    
    Returns:
        X_train, X_test, y_train, y_test: arrays
            Split input data and target variables into train, validation and test sets.
    """
    assert val_size + test_size < 1.0, "The sum of val_size and test_size must be less than 1.0"
    n_samples = len(X)
    for y in ys:
        if len(y) != n_samples:
            raise ValueError("Number of samples in X and y arrays must be the same.")

    if shuffle:
        if random_state is not None:
            np.random.seed(random_state)
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        X = X[indices]
        for i in range(len(ys)):
            ys[i] = ys[i][indices]

    train_size = 1.0 - val_size - test_size
    train_end = int(train_size*n_samples)
    val_end = train_end + int(val_size*n_samples)
    
    X_train = X[:train_end]
    y_train = [y[:train_end] for y in ys]
    X_val = X[train_end:val_end]
    y_val = [y[train_end:val_end] for y in ys]
    X_test = X[val_end:]
    y_test = [y[val_end:] for y in ys]

    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
X_data, y_data = df_to_X_y_data(concatenated_df, window_size=PREDICTION_WINDOW_SIZE)
print("X shape:", X_data.shape)
print("y shape:", len(y_data), y_data[0].shape, y_data[1].shape)

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = multi_task_train_val_test_split(X_data, y_data, val_size=0.1, test_size=0.1, random_state=RANDOM_SEED)
print("X shape (train, validation, test):", X_train.shape, X_val.shape, X_test.shape)
print("y shape (train, validation, test):", len(y_train), y_train[0].shape, y_train[1].shape, len(y_val), y_val[0].shape, y_val[1].shape, len(y_test), y_test[0].shape, y_test[1].shape)

In [ ]:
class MultiTaskDataset(Dataset):
    def __init__(self, X_data, y1_data, y2_data):
        self.X_data = X_data
        self.y1_data = y1_data
        self.y2_data = y2_data
        
    def __len__(self):
        return len(self.X_data)
    
    def __getitem__(self, idx):    
        return self.X_data[idx], self.y1_data[idx], self.y2_data[idx]

In [ ]:
class LSTMMultiTaskClassifier(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes1, num_classes2, dropout=0.2):
        super(LSTMMultiTaskClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, num_classes1)
        self.fc2 = nn.Linear(hidden_size, num_classes2)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out1 = self.fc1(out)
        out2 = self.fc2(out)
        return out1, out2

In [ ]:
# Model definition
input_size = X_train.shape[2]
sequence_length = X_train.shape[1]
initial_learning_rate = 0.01
num_layers = 3
hidden_size = 64

model = LSTMMultiTaskClassifier(input_size, hidden_size, num_layers, num_classes_actions, num_classes_objects).to(device)

# Loss function and optimizer
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
weight_task1 = 1.0
weight_task2 = 0.5
optimizer = optim.Adam(model.parameters(), lr=initial_learning_rate)
scheduler = MultiStepLR(optimizer, milestones=[20, 40], gamma=0.1)

summary(model, input_size=(16, sequence_length, input_size))

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

y1_train_tensor = torch.tensor(y_train[0], dtype=torch.float32).to(device)
y1_val_tensor = torch.tensor(y_val[0], dtype=torch.float32).to(device)
y1_test_tensor = torch.tensor(y_test[0], dtype=torch.float32).to(device)

y2_train_tensor = torch.tensor(y_train[1], dtype=torch.float32).to(device)
y2_val_tensor = torch.tensor(y_val[1], dtype=torch.float32).to(device)
y2_test_tensor = torch.tensor(y_test[1], dtype=torch.float32).to(device)

In [ ]:
# Training loop
epochs = 100
batch_size = 16

# Early stopping parameters
early_stopping_enabled = True
patience = 15
min_delta_improvement = 0.001
restore_best_weights = False

best_loss = np.Inf
counter = 0
best_epoch = 0

if early_stopping_enabled and restore_best_weights:
    best_model_weights = copy.deepcopy(model.state_dict())

train_losses = []
val_losses = []

train_loader = DataLoader(MultiTaskDataset(X_train_tensor, y1_train_tensor, y2_train_tensor), batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y1_batch, y2_batch in train_loader:
        X_batch = X_batch.to(device)
        y1_batch = y1_batch.to(device)
        y2_batch = y2_batch.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs1, outputs2 = model(X_batch)
        loss1 = criterion1(outputs1, y1_batch)
        loss2 = criterion2(outputs2, y2_batch)
        total_loss = weight_task1*loss1 + weight_task2*loss2
        
        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()
        train_loss += total_loss.item()

    # Learning rate scheduler step
    scheduler.step()

    # Validation loss
    model.eval()
    with torch.no_grad():
        val_outputs1, val_outputs2 = model(X_val_tensor)
        val_loss1 = criterion1(val_outputs1, y1_val_tensor)
        val_loss2 = criterion2(val_outputs2, y2_val_tensor)
        total_val_loss = weight_task1*val_loss1 + weight_task2*val_loss2

    train_losses.append(train_loss/len(train_loader))
    val_losses.append(total_val_loss.item())
    
    print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {total_val_loss:.4f}, lr: {optimizer.param_groups[0]['lr']}")

    # Early stopping logic
    if early_stopping_enabled:
        if total_val_loss < best_loss - min_delta_improvement:
            best_loss = total_val_loss
            counter = 0
            best_epoch = epoch + 1
            if restore_best_weights:
                best_model_weights = copy.deepcopy(model.state_dict())
        else:
            counter += 1
        
        if counter >= patience:
            restore_best_weights_message = f" Best weights restored to epoch {best_epoch}." if restore_best_weights else ""
            print(f"Early stopping at epoch {epoch + 1} with no improvement in validation loss.{restore_best_weights_message}")
            break

# Restore best weights
if early_stopping_enabled and restore_best_weights:
    model.load_state_dict(best_model_weights)

In [ ]:
# Save model state and optimizer state
model_checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
torch.save(model_checkpoint, "model_ap.pth")

In [ ]:
# Load model state and optimizer state
model_checkpoint = torch.load("model_ap.pth")
model.load_state_dict(model_checkpoint["state_dict"])
optimizer.load_state_dict(model_checkpoint["optimizer"])

In [ ]:
# Save model TorchScript
model_scripted = torch.jit.script(model)
model_scripted.save(os.path.join("weights", "model_ap.pt"))

In [ ]:
# Learning curves
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Total Loss (Weighted Sum of Cross-Entropy Losses)")
plt.legend()
plt.title("Learning Curves")
plt.show()

In [ ]:
def plot_confusion_matrix(cm, class_names, normalize=False):
    if normalize:
        cm = cm/cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10, 8))
    sns.heatmap(np.transpose(cm), annot=True, fmt='.2f' if normalize else 'd', cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("True")
    plt.ylabel("Predicted")
    plt.title("Confusion Matrix")
    plt.show()

def evaluate_performance_metrics(y_true, outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    
    accuracy = accuracy_score(y_true, preds)
    precision = precision_score(y_true, preds, average="macro")
    recall = recall_score(y_true, preds, average="macro")
    f1 = f1_score(y_true, preds, average="macro")
    conf_matrix = confusion_matrix(y_true, preds)
    roc_auc = roc_auc_score(y_true, outputs.cpu().numpy(), multi_class="ovr")

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    # print(f"Confusion Matrix:\n{conf_matrix}")
    plot_confusion_matrix(conf_matrix, labels, normalize=True)

def ROC_curve(y_true, outputs, labels):
    fpr = {}
    tpr = {}
    roc_auc = {}
    n_classes = outputs.size(1)
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true == i, outputs[:, i].cpu().numpy())
        roc_auc[i] = roc_auc_score(y_true == i, outputs[:, i].cpu().numpy())

    plt.figure()
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], lw=2, label=f"ROC curve of action '{labels[i]}' (area = {roc_auc[i]:.2f})")

    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC) Curve")
    plt.legend(loc="lower left", bbox_to_anchor=(0.4, 0.0))
    ax = plt.gca()
    ax.set_aspect("equal", adjustable="box")
    plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    val_outputs1, val_outputs2 = model(X_val_tensor)
    val_outputs1 = F.softmax(val_outputs1, dim=1)
    val_outputs2 = F.softmax(val_outputs2, dim=1)
    test_outputs1, test_outputs2 = model(X_test_tensor)
    test_outputs1 = F.softmax(test_outputs1, dim=1)
    test_outputs2 = F.softmax(test_outputs2, dim=1)

val_true1 = torch.argmax(y1_val_tensor, dim=1).cpu().numpy()
val_true2 = torch.argmax(y2_val_tensor, dim=1).cpu().numpy()
test_true1 = torch.argmax(y1_test_tensor, dim=1).cpu().numpy()
test_true2 = torch.argmax(y2_test_tensor, dim=1).cpu().numpy()

In [ ]:
test_pred1 = torch.argmax(test_outputs1, axis=1).tolist()
test_pred2 = torch.argmax(test_outputs2, axis=1).tolist()
conf1 = torch.max(test_outputs1, axis=1).values.tolist()
conf2 = torch.max(test_outputs2, axis=1).values.tolist()

print("True - Predicted - Probability", end="\n\n")
for i in range(test_outputs1.shape[0]):
    true_object_index = test_true2[i]
    true_object = OBJECT_NAMES[true_object_index] if true_object_index < len(OBJECT_NAMES) else "none"
    predicted_object_index = test_pred2[i]
    predicted_object = OBJECT_NAMES[predicted_object_index] if predicted_object_index < len(OBJECT_NAMES) else "none"

    print(f"{Action(test_true1[i]).name.lower()} {true_object} - {Action(test_pred1[i]).name.lower()} {predicted_object} - {conf1[i]:.3f} {conf2[i]:.3f}")

In [ ]:
# Test set performance metrics - Actions
evaluate_performance_metrics(test_true1, test_outputs1, [action.name.lower() for action in list(Action)])

In [ ]:
# Test set performance metrics - Objects
evaluate_performance_metrics(test_true2, test_outputs2, list(OBJECT_NAMES.values()) + ["none"])

In [ ]:
# Test set ROC curve - Actions
ROC_curve(test_true1, test_outputs1, [action.name.lower() for action in list(Action)])

In [ ]:
# Test set ROC curve - Objects
ROC_curve(test_true2, test_outputs2, list(OBJECT_NAMES.values()) + ["none"])

In [ ]:
sequence_data = np.array((
    (1, 7),
    (2, 7),
    (1, 0),
    (2, 0),
    (1, 0)
))

In [ ]:
one_hot_sequence_data = deque(maxlen=PREDICTION_WINDOW_SIZE)
for row in sequence_data:
    one_hot_sequence_data.append(np.concatenate((one_hot_encode_class(row[0], num_classes_actions), one_hot_encode_class(row[1], num_classes_objects)), axis=0))

model.eval()
X = torch.tensor(np.array(one_hot_sequence_data), dtype=torch.float32).to(device)
X = torch.unsqueeze(X, axis=0)
with torch.no_grad():
    outputs1, outputs2 = model(X)
    prob1 = F.softmax(outputs1, dim=1)[0]
    prob2 = F.softmax(outputs2, dim=1)[0]
print("Action probabilities:")
for action_index, action_prob in enumerate(prob1):
    print(f"- {Action(action_index).name.lower()}: {(action_prob*100):.2f} %")
action_prediction = Action(torch.argmax(prob1).item()).name.lower()
print("Action prediction:", action_prediction, end="\n\n")
print("Object probabilities:")
for object_index, object_prob in enumerate(prob2):
    object_name = OBJECT_NAMES[object_index] if object_index < len(OBJECT_NAMES) else "none"
    print(f"- {object_name}: {(object_prob*100):.2f} %")
predicted_object_index = torch.argmax(prob2).item()
predicted_object = OBJECT_NAMES[predicted_object_index] if predicted_object_index < len(OBJECT_NAMES) else "none"
print("Object prediction:", predicted_object)

In [ ]:
def predict_next_action(one_hot_sequence_data):
    """
    Predict the next most probable action based on provided one-hot sequence data.
    Returns the probabilities for predicted actions and objects.
    """
    X = torch.tensor(np.array(one_hot_sequence_data), dtype=torch.float32).to(device)
    X = torch.unsqueeze(X, axis=0)
    with torch.no_grad():
        outputs1, outputs2 = model(X)
        prob1 = F.softmax(outputs1, dim=1)[0]
        prob2 = F.softmax(outputs2, dim=1)[0]
        return prob1, prob2

def search_for_future_action(sequence_data, action_to_find, force_object=False):
    """
    Uses action prediction to search for a specified future action.
    
    Parameters:
        sequence_data: numpy.ndarray, shape (sequence_length, 2)
            Sequence data in pairs (tuple): (action_index, object_index).
        action_to_find: Action
            Action to search for in the future.
        force_object: bool, default False
            If True, if the object found for the specified action is none,
            the second most probable object will be returned.
    
    Returns:
        prediction_pairs: list
            List of predicted pairs from the next step to the action to find.
        conf: tuple (action_confidence, object_confidence)
            Confidence in the action to be found and the relative predicted object,
            calculated as the conditional probability of the prediction_pairs elements (multiplied confidences).
        object_forced: bool
            Whether the object found for the specified action was forced.
            It always returns False if force_object is False.
    """
    one_hot_sequence_data = deque(maxlen=PREDICTION_WINDOW_SIZE)
    for action_pair in sequence_data:
        one_hot_sequence_data.append(np.concatenate((one_hot_encode_class(action_pair[0], num_classes_actions), one_hot_encode_class(action_pair[1], num_classes_objects)), axis=0))

    prediction_pairs = []
    confidences = []
    object_forced = False
    while True:
        prob1, prob2 = predict_next_action(one_hot_sequence_data)
        action_prediction_index = torch.argmax(prob1).item()
        object_prediction_indices = torch.sort(prob2, descending=True).indices.tolist()
        
        if Action(action_prediction_index) == action_to_find: # Action found
            if force_object and object_prediction_indices[0] == len(OBJECT_NAMES): # Object forced
                prediction_pairs.append((action_prediction_index, object_prediction_indices[1]))
                confidences.append((prob1[action_prediction_index].item(), prob2[object_prediction_indices[1]].item()))
                object_forced = True
            else: # Object not forced
                prediction_pairs.append((action_prediction_index, object_prediction_indices[0]))
                confidences.append((prob1[action_prediction_index].item(), prob2[object_prediction_indices[0]].item()))
            break
        else: # Add predicted row to sequence and predict next
            action_pair = (action_prediction_index, object_prediction_indices[0])
            prediction_pairs.append(action_pair)
            confidences.append((prob1[action_pair[0]].item(), prob2[action_pair[1]].item()))
            one_hot_sequence_data.append(np.concatenate((one_hot_encode_class(action_pair[0], num_classes_actions), one_hot_encode_class(action_pair[1], num_classes_objects)), axis=0))

    return prediction_pairs, tuple(np.prod(np.array(confidences), axis=0)), object_forced

In [ ]:
prediction_pairs, conf, object_forced = search_for_future_action(sequence_data, Action.PICK, force_object=True)
print("Prediction", prediction_pairs, conf, object_forced)